In [1]:
import pandas as pd
import os
import time
import logging
from datetime import date

In [2]:
from googletrans import Translator

In [3]:
# # 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)

In [4]:
proxy = 'http://127.0.0.1:1080'
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [5]:
# translator = Translator(service_urls=[
#       'translate.google.com',
#       'translate.google.co.kr',
#     ])
# translator.translate('韩国', dest='ko').text

In [6]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [7]:
folder_path = '../shopee_product_translation/'
tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'tengus_tw_product_list.xlsx'))
# tw_site_product_list = pd.read_excel(os.path.join(folder_path, 'translated_product_2019_10_19.xlsx'))
# tw_site_product_list = tw_site_product_list[tw_site_product_list['是否翻译'] == 'no']
# tw_site_product_list.head(5)

In [8]:
# tw_site_product_list.columns

In [9]:
# tw_site# 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)_product_list = tw_site_product_list[tw_site_product_list['产品id'] == 2496637042]

In [10]:
# top_cat_in_tw = tw_site_product_list[['分类ID', '产品id']].groupby('分类ID').agg({'产品id': 'nunique'}).reset_index().sort_values(by='产品id', ascending=False).reset_index().head(5)
# top_cat_in_tw

In [11]:
# top_cat_id = top_cat_in_tw['分类ID'][0]
# top_cat_id

In [12]:
output_excel_columns = [
                        '变种名称',
                        '变种属性名称一',
                        '变种属性名称二',
                        '变种属性值一',
                        '变种属性值二',
                        '*价格（必填）',
                        '促销价',
                        '折扣活动ID',
                        '*库存（必填）',
                        '*重量(kg)(必填）',
                        '*主图（URL）地址（必填）',
                        '附图1',
                        '附图2',
                        '附图3',
                        '附图4',
                        '附图5',
                        '附图6',
                        '附图7',
                        '附图8',
                        '变种图',
                        '长(cm)',
                        '宽(cm)',
                        '高(cm)',
                        '发货期',
                        '来源URL',
                        '尺码图'
]

In [13]:
tw_site_product_list_selected_columns = ['分类ID', '产品属性', 'Parent SKU', '产品标题', '产品描述', 'sku', '变种名称', '变种属性名称一',
       '变种属性名称二', '变种属性值一', '变种属性值二', '价格', '促销价', '折扣活动ID', '库存', '重量', '主图（URL）地址', '附图1',
       '附图2', '附图3', '附图4', '附图5', '附图6', '附图7', '附图8地址', '变种图', '长（cm）',
       '宽（cm）', '高（cm）', '发货期', '来源URL', '尺码图']

In [14]:
tw_site_product_list_with_selected_columns = tw_site_product_list[tw_site_product_list_selected_columns]
# tw_site_product_list_with_selected_columns.head(5)

In [15]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[tw_site_product_list_with_selected_columns['分类ID'] == top_cat_id].reset_index()
# tw_site_product_list_with_selected_columns

In [16]:
# 把1688的来源url拆出来，把productid填充为parent sku
tw_site_product_list_with_selected_columns['Parent SKU'] = tw_site_product_list_with_selected_columns['来源URL'].str.split("/", n = 4, expand = True)[4].str.split(".", n = 1, expand=True)[0]
# tw_site_product_list_with_selected_columns.head(5)

C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [17]:
# remove emoji
import re

emoji_pattern = re.compile(
                u'['
                u'\U0001F300-\U0001F64F'
                u'\U0001F680-\U0001F6FF'
                u'\u2600-\u2B55'
                u'\u23cf'
                u'\u23e9'
                u'\u231a'
                u'\u3030'
                u'\ufe0f'
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u'\U00010000-\U0010ffff'
                u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
                u'\U00002702-\U000027B0]+', flags=re.UNICODE)

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

# remove_emoji('ins爆款 🥕胡蘿卜🥕寶寶針織竹節棉爬服 嬰兒兔裝 嬰兒連身衣 寶寶包屁衣連身衣 中間開襟包屁衣 寶寶連體衣哈衣爬服')

In [18]:
tw_site_product_list_with_selected_columns['产品标题'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品标题']]
tw_site_product_list_with_selected_columns['产品描述'] = [remove_emoji(x) for x in tw_site_product_list_with_selected_columns['产品描述']]

C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
# 去掉某些字眼
def remove_words(text, word):
    return text.replace(word, "")

# remove_words('爆款哈哈ha', '爆款')

In [20]:
# 需要去除的字眼
words_to_remove = ['爆款', 'Aliexpress', 'AliExpres',
                   'Wish', 'wish', 'Ebay', 'ebay',
                   'amazon', 'Amazon', '速卖通' ,
                   '2019', '2018', '一件代发',
                   '代发', '批发', 'Ins', 'INS', 'ins']

for word in words_to_remove:
    tw_site_product_list_with_selected_columns['产品标题'] = [remove_words(x, word) for x in tw_site_product_list_with_selected_columns['产品标题']]
    tw_site_product_list_with_selected_columns['产品描述'] = [remove_words(x, word) for x in tw_site_product_list_with_selected_columns['产品描述']]

C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
# 物流属性
tw_site_product_list_with_selected_columns['长（cm）'] = tw_site_product_list_with_selected_columns['长（cm）'].fillna(18)
tw_site_product_list_with_selected_columns['宽（cm）'] = tw_site_product_list_with_selected_columns['宽（cm）'].fillna(13)
tw_site_product_list_with_selected_columns['高（cm）'] = tw_site_product_list_with_selected_columns['高（cm）'].fillna(5)
tw_site_product_list_with_selected_columns['发货期'] = tw_site_product_list_with_selected_columns['发货期'].fillna(2)

C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [22]:
set(tw_site_product_list_with_selected_columns['变种属性名称一'].tolist())

{'颜色'}

In [23]:
set(tw_site_product_list_with_selected_columns['变种属性名称二'].tolist())

{nan, '适合身高'}

In [24]:
set(tw_site_product_list_with_selected_columns['变种属性值一'].tolist())

{'1498杏',
 '秋季电灯',
 '小恐龙',
 '5500炫酷黑',
 '春秋我爱妈妈开裆',
 '2168小猪',
 '39150',
 '1529紫色',
 'DTZ9275N粉色爱心毛圈款',
 'HY2237T',
 '秋季小蓝猫',
 '1541蓝色',
 '棕熊',
 'HY2424R玫红色',
 'XTZ8288N条纹7',
 'HY2582Y黄色',
 '民族风蓝色',
 '网眼-薄款中长袖-绿仙人掌',
 '春秋我爸超帅',
 'HY2696R花儿',
 '1498蓝',
 'P米色熊',
 '1890蓝',
 '1548黄',
 'LTY362LOVE',
 'HY2566W波点',
 'M汽车',
 '菠萝',
 'DTZ9462B蓝色',
 '5323小鸟',
 '淑女果（黄色）',
 '39126',
 'RS726偏开小果园',
 '2203',
 'HY2265W-尺子',
 'H855紫碎花',
 '小棕熊',
 '3355两用裆 浅咖色',
 'RS759星星宝',
 'HY2232N黑色',
 'HY2594N黑橙',
 '无袖黑色套装',
 'HY2429G绿柠檬',
 'HY2524P红灯笼',
 'HY2568B小马',
 '黑色 无袖',
 '5411粉色',
 'XTZ8280B蓝色',
 'DTZ9462N黑+红',
 '绅士小礼帽',
 '5503苹果鸭',
 '女随机长裤',
 'HY2232Z紫色',
 '5431海狮',
 '春秋面包超人套头',
 'HY2598Y黄色',
 'DTZ9473R',
 '1541粉',
 'RS766A哒哒鸭',
 'LTY362小可爱',
 '粉红',
 '短袖小红狐',
 '1878绿',
 '夹绒滑板熊',
 'HY2318W线条',
 'LTY362藏青小猫',
 '5516爱心',
 'LTY1938恐龙',
 'HY2243B蓝黑白',
 '字母熊',
 'HY2124B仙人掌',
 '5511三只猫',
 '39128',
 'HY2619G熊',
 '精梳棉-长袖哈衣-小蜜蜂',
 '39118',
 'H855绿太阳花',
 'B82',
 '直条棉领结粉色',
 '5421粉色',
 '直条棉领结小绅士',
 '灰色长袖',
 '小彩虹',
 'HY

In [25]:
set(tw_site_product_list_with_selected_columns['变种属性值二'].tolist())

{nan,
 '110CM建议体重36斤内',
 '100',
 '90cm/11-14个月（10-12kg）',
 '73CM',
 '18M',
 '73cm',
 '110cm',
 '95',
 '6m',
 '80cm-120cm/1手5件',
 '150cm',
 '130适合120到130CM',
 '80~110cm/1手4件',
 '100适合90到100CM',
 '59',
 '80码（L码）',
 '66码 身高59-65cm左右 体重12-15斤左右',
 '18M（85cm）',
 '4',
 '5T',
 '140适合130到140',
 '140',
 '100码（XXL码）',
 '3Y',
 '9M',
 '100cm',
 '整手混批240件以上',
 '140cm',
 '7T',
 '9m',
 '90码（XL码）',
 '66cm/3-6个月（6-7kg）',
 '36M（95cm）',
 '100码 身高90-100cm左右 体重30斤以内',
 '4T',
 '6',
 '90cm',
 '95CM',
 '80cm/9-11个月（8-9kg）',
 '100CM',
 '24M',
 '12M',
 '60cm',
 '1M（52cm）',
 '90',
 '18-24M',
 '3T',
 '6T',
 '80CM建议体重23斤内',
 '2T',
 '18m',
 '6-24m=5件/手 拍5的倍数',
 '130CM',
 '整手混批120件以上',
 '73CM-130CM/1手7件[请拍7次]',
 '130cm',
 '70cm',
 '60',
 '2Y',
 '70',
 '95cm',
 '80CM',
 '3M',
 '80码 身高73-79cm左右 体重17-20斤左右',
 '9M（73cm）',
 '110',
 '110适合100到110CM',
 '73',
 '110码（3xl码）',
 '8T',
 '5',
 '5Y',
 '9',
 '52cm',
 '80cm',
 '6M',
 '4Y',
 '120CM',
 '12-18M',
 '110CM',
 '150',
 '3',
 '120CM建议体重40斤内',
 '90CM',
 '6M（66cm）',
 '120',
 

In [26]:
# translate_currency = 458.61
# profit_rate = 1.2
# tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * translate_currency / profit_rate

In [27]:
# tw_site_product_list_with_selected_columns.head(5)

In [28]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])
# translator.translate('韩国', dest='ko').text

In [29]:
# translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
# for translation in translations:
#     print(translation.origin, ' -> ', translation.text)

In [30]:
translate_columns = ['产品标题', '产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']
# translate_columns = ['产品标题']
# translate_columns = ['产品描述', '变种名称', '变种属性名称一', '变种属性名称二', '变种属性值一', '变种属性值二']

In [31]:
# 把空值填充为string格式
tw_site_product_list_with_selected_columns[translate_columns] = tw_site_product_list_with_selected_columns[translate_columns].fillna(value='不需翻译')

C:\Users\kuang\Anaconda3\lib\site-packages\pandas\core\frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [32]:
# 把variation_name做一些删减
variation_list = set(list(tw_site_product_list_with_selected_columns['变种属性名称一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性名称二'].unique()))
logging.info('variation list: ' + str(variation_list))

 2019-12-15 23:22:50,485 - INFO - variation list: {'不需翻译', '颜色', '适合身高'}


In [33]:
variation_rename = {'适合身高': '身高',
                    '童袜尺码': '尺码',
                    '不需翻译': 'not set',
                    '73CM': '73cm',
                    '100cm 少量现货': '100cm',
                    '100CM建议体重32斤内': '100cm',
                    '110CM建议体重36斤内': '110cm',
                    '120CM建议体重40斤内': '120cm',
                    '73CM建议体重18斤内': '73cm',
                    '80CM建议体重23斤内': '80cm',
                    '90CM建议体重28斤内': '90cm',
                    '编码': 'no size',
                    'XXL码（建议身高130-150cm）':'XXL',
                    'L码 （建议身高100-100cm）':'L ',
                    'XL码(建议身高115-130cm)':'XL',
                    '20/21（实际内长12.4cm）':'20/21',
                    '22/23（实际内长13.6cm）':'22/23',
                    '24/25（实际内长14.7cm）':'24/25',
                    '26/27（实际内长15.9CM）':'26/27',
                    '28/29（实际内长17cm）':'28/29',
                    '袋装（加大 6-18个月）':'6-18月',
                    '袋装（常规 0-12个月）':'0-12月',
                    'M码（2-4岁 防滑底）':'M',
                    'S码（0-2岁 防滑底）':'S',
                    'M（1-3岁 防滑底）':'M',
                    'M（2-4岁 防滑底）':'M',
                    'S（0-1岁 防滑底）':'S',
                    'S（0-2岁 防滑底）':'S',
                    'S (6-12)个月':'S ',
                    'S （6-12个月）':'S ',
                    'XS( 0-6个)月':'XS',
                    '12-24个月闭档':'12-24月',
                    '32*12.5cm':'32*12.5cm',
                    'L码24-36个月':'L',
                    'XL码 9-12岁':'XL',
                    'XL码（5-8岁）':'XL',
                    'XS（0-6个月）':'XS',
                    '6-12个月闭档':'6-12月',
                    'L码 7-10岁':'L',
                    'L码（3-5岁）':'L',
                    'L码（7-9岁）':'L',
                    'M （1-3）岁':'M ',
                    'M码（1-3岁）':'M',
                    'M码（4-6岁）':'M',
                    'S码（0-1岁）':'S',
                    'S码（1-3岁）':'S',
                    'XL码 5-8岁':'XL',
                    'XL码9-12岁':'XL',
                    '（0-1岁）均码':'0-1岁',
                    'L码 3-5岁':'L',
                    'L码 5-8岁':'L',
                    'L（3-5岁）':'L',
                    'M码 1-3岁':'M',
                    'M码 3-5岁':'M',
                    'M码 4-6岁':'M',
                    'M（1-3岁）':'M',
                    'S码 0-1岁':'S',
                    'S码 1-3岁':'S',
                    '（1-3岁）均码':'1-3岁',
                    '12*9CM':'12*9CM',
                    '12-24月':'12-24月',
                    '2-4岁闭档':'2-4岁',
                    '4-6岁闭档':'4-6岁',
                    '6-12个月':'6-12月',
                    '6-18个月':'6-18月',
                    'L码3-5岁':'L',
                    'L码4-6岁':'L',
                    'L码5-8岁':'L',
                    'L码6-8岁':'L',
                    'M码1-3岁':'M',
                    'M码2-4岁':'M',
                    'M码3-5岁':'M',
                    'S码0-1岁':'S',
                    'S码0-2岁':'S',
                    'S码1-3岁':'S',
                    '0-2岁':'0-2岁',
                    '1-3岁':'1-3岁',
                    '11CM':'11CM',
                    '11cm':'11cm',
                    '12CM':'12CM',
                    '12cm':'12cm',
                    '13CM':'13CM',
                    '13cm':'13cm',
                    '14cm':'14cm',
                    '2-4岁':'2-4岁',
                    '28CM':'28CM',
                    '28cm':'28cm',
                    '3-5岁':'3-5岁',
                    '35CM':'35CM',
                    '35cm':'35cm',
                    '4-6岁':'4-6岁',
                    '45CM':'45CM',
                    '45cm':'45cm',
                    '5-8岁':'5-8岁',
                    '均码':'均码'
}
tw_site_product_list_with_selected_columns['变种属性名称一'] = tw_site_product_list_with_selected_columns['变种属性名称一'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性名称二'] = tw_site_product_list_with_selected_columns['变种属性名称二'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['变种属性值一'].apply(lambda x: variation_rename[x] if x in variation_rename else x)
tw_site_product_list_with_selected_columns['变种属性值二'] = tw_site_product_list_with_selected_columns['变种属性值二'].apply(lambda x: variation_rename[x] if x in variation_rename else x)

C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [34]:
# 给变种属性值一、二做适当缩减
# 先把索引列取出来
tw_site_product_list_with_selected_columns['index'] = tw_site_product_list_with_selected_columns.index
tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['index'].groupby([tw_site_product_list_with_selected_columns['Parent SKU'],tw_site_product_list_with_selected_columns['变种属性值二']]).rank(ascending=1,method='dense')

tw_site_product_list_with_selected_columns['变种属性值一'] = tw_site_product_list_with_selected_columns['变种属性值一'].apply(str)

C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [35]:
tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns.drop(['index'], axis=1)
# tw_site_product_list_with_selected_columns.head(5)

In [36]:
# 国家&折扣id
country_info = {'th': 1035824326,
                'id': 1040198485,
                'sg': 1052622558}
# country_info['th']

# 国家&需藏运费
country_logistics_info = {'th': 40,
                          'id': 12800,
                          'sg': 3.25}

# 语言
translated_country_name = {'th': 'th',
                           'id': 'id',
                           'sg': 'en'}

In [37]:
# 选择国家
selected_country = 'sg'

# 藏运费+各种费率，系数1.28
tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] + country_logistics_info[selected_country]
tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * 1.28

# 价格 折扣价
tw_site_product_list_with_selected_columns['促销价'] = tw_site_product_list_with_selected_columns['价格']

# 印尼不能折1折
tw_site_product_list_with_selected_columns['价格'] = tw_site_product_list_with_selected_columns['价格'] * 10
tw_site_product_list_with_selected_columns['折扣活动ID'] = country_info[selected_country]

# 重量
tw_site_product_list_with_selected_columns['重量'] = 0.3

# tw_site_product_list_with_selected_columns.head(5)

In [38]:
tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'to_be_translated_product.xlsx'), index=False)

C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/851/680/12591086158_1282123135.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/639/413/12555314936_1237798930.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/385/932/12430239583_137037315.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/

C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/285/282/10674282582_1522398534.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/536/224/10740422635_2143218616.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/479/393/10745393974_137037315.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/

C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/902/998/11742899209_2143218616.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/573/029/11742920375_2143218616.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank/2019/407/475/11208574704_2143218616.jpg' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://cbu01.alicdn.com/img/ibank

C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/40432815681.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/40468595803.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/40500809895.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/520809225918.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,53

C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/586506539742.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/586539221257.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/586611048450.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\kuang\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:837: UserWarning: Ignoring URL 'https://detail.1688.com/offer/586809043922.html' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65

In [39]:
# 对variation_value做一些删减
# variation_value_list = set(list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()) + list(tw_site_product_list_with_selected_columns['变种属性值一'].unique()))
# logging.info('variation value list: ' + str(variation_value_list))

In [40]:
# tw_site_product_list_with_selected_columns = tw_site_product_list_with_selected_columns[:5]
# # tw_site_product_list_with_selected_columns

In [41]:
today = date.today()
d1 = today.strftime("%Y_%m_%d")

In [474]:
# for index, row in tw_site_product_list_with_selected_columns.iterrows():
#     try:
#         list_to_translate = []
#         for i, value in enumerate(translate_columns):
#             list_to_translate.append(row[translate_columns[i]])
        
#         translate_language = 'en'
#         translator = Translator(service_urls=['translate.google.com'])
#         list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
#         # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
#         time.sleep(10)
        
#         i = 0
#         for translate_language in list_be_translated:
#             tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
#             i += 1
        
#         if (index + 1) % 5 == 0:
#             logging.info(str(index + 1) + ' rows have been translated.')
            
#         tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_' + d1 + '.xlsx'), index=False)
#     except Exception as err:
#         logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err))
#         pass

In [ ]:
for index, row in tw_site_product_list_with_selected_columns.iterrows():
    translated = 0
    while translated == 0:
        try:
            list_to_translate = []
            for i, value in enumerate(translate_columns):
                list_to_translate.append(row[translate_columns[i]])

            # translate_language = 'id'
            translate_language = translated_country_name[selected_country]
            translator = Translator(service_urls=['translate.google.com'])
            list_be_translated = translator.translate(list_to_translate, dest=translate_language, src='zh-CN')
            # list_be_translated = translator.translate(list_to_translate, dest=translate_language)
            time.sleep(10)

            i = 0
            for translate_language in list_be_translated:
                tw_site_product_list_with_selected_columns.at[index, translate_columns[i]] = translate_language.text
                i += 1

            if (index + 1) % 100 == 0:
                logging.info(str(index + 1) + ' rows have been translated.')

            tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'translated_product_' + selected_country + '_' + d1 + '.xlsx'), index=False)
            translated = 1
        except Exception as err:
            logging.warning(str(index + 1) + ' rows have not been translated. Error message: ' + str(err) + ', try again.')
            pass        

 2019-12-11 12:30:24,218 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:24,282 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:24,312 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:24,343 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-12-11 12:30:25,125 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:25,152 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:25,175 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:25,199 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-12-11 12:30:26,054 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:26,078 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:26,105 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:26,128 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-12-11 12:30:27,141 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:27,234 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:27,340 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:27,420 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

 2019-12-11 12:30:28,492 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:28,531 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:28,574 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-11 12:30:28,594 - WARNING - 68 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url

...

 2019-12-12 20:08:34,054 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:34,082 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:34,131 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:34,827 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-12 20:08:37,164 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:37,819 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:37,892 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:37,929 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-12 20:08:43,093 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:43,125 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:43,164 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:43,199 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-12 20:08:46,168 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:46,199 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:46,862 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:46,896 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

 2019-12-12 20:08:49,007 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:49,043 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:49,078 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))), try again.
 2019-12-12 20:08:49,101 - WARNING - 3887 rows have not been translated. Error message: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded 

In [ ]:
# 优化结果
output_path = os.path.join(folder_path, 'translated_product_' + selected_country + '_' + d1 + '.xlsx')
dianxiaomi_product_list = pd.read_excel(output_path)

dianxiaomi_product_list['产品标题'] = dianxiaomi_product_list['产品标题'].apply(lambda x: x[:100])

variation_name_key_value = {
    'ไม่จำเป็นต้องแปล': 'size',
    'ไม่ได้ตั้งค่า': 'size',
    'เหมาะสำหรับความสูง': 'ความสูง',
    'ข้อมูลจำเพาะ (กว้าง * ยาวซม.)': 'size',
    'tidak diatur': 'size'
}

dianxiaomi_product_list['变种属性名称二'] = dianxiaomi_product_list['变种属性名称二'].apply(lambda x: variation_name_key_value[x] if x in variation_name_key_value else x)

In [ ]:
# dianxiaomi_product_list['index'] = dianxiaomi_product_list.index
# dianxiaomi_product_list['变种属性值一'] = dianxiaomi_product_list['index'].groupby([dianxiaomi_product_list['Parent SKU'],dianxiaomi_product_list['变种属性值二']]).rank(ascending=1,method='dense')

# dianxiaomi_product_list['变种属性值一'] = dianxiaomi_product_list['变种属性值一'].apply(str)

In [ ]:
dianxiaomi_product_list.to_excel(os.path.join(folder_path, 'translated_product_' + selected_country + '_' + d1 + '_edited(1).xlsx'), index=False)

In [ ]:
# for translate_column in translate_columns:
#     # tw_site_product_list_with_selected_columns[translate_column] = tw_site_product_list_with_selected_columns[translate_column].fillna(0)
#     # tw_site_product_list_with_selected_columns[translate_column]  = [translator.translate(x, dest=translate_language).text 
#                                                                      # for x in tw_site_product_list_with_selected_columns[translate_column]]
#     for index, row in tw_site_product_list_with_selected_columns.iterrows():
#         try:
#             orginal_text = tw_site_product_list_with_selected_columns.at[index, translate_column]
#             tw_site_product_list_with_selected_columns.at[index, translate_column] = translator.translate(row[translate_column], dest=translate_language).text
#             # print(row[translate_column])
#             logging.info(translate_column + ' - ' + str(index + 1) + ' rows have been translated.')
#             # logging.info(orginal_text + ' -> ' + tw_site_product_list_with_selected_columns.at[index, translate_column])
#             time.sleep(5)
#         except:
#             logging.warning(translate_column + ' - ' + str(index + 1) + ' rows have not been translated.')
#             pass
        
#     # print(tw_site_product_list_with_selected_columns.head(5))
    
#     tw_site_product_list_with_selected_columns.to_excel(os.path.join(folder_path, 'tw_translated_product(1).xlsx'), index=False)
#     logging.info(str(translate_column) + ' is done.')